In [ ]:
import math
import scipy
import numpy as np
import pandas as pd

# Import relevant modSAR classes 
import modSAR
from modSAR.network_algorithms import ModSAR
from modSAR.dataset import QSARDataset, QSARDatasetIO

# plotnine is the python version of ggplot2
from plotnine import *

import warnings
warnings.filterwarnings("ignore")

from rdkit import Chem
from rdkit.Chem import AllChem, Draw

from rdkit import RDLogger

import rdkit.Geometry
from rdkit.Chem import rdFMCS, PandasTools
from rdkit.Chem.Draw import IPythonConsole 
from rdkit.Chem import PandasTools

from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV

# 2.5uM in Log units
CUTOFF_ACTIVITY = - np.log10(2.5e-06)

s4_template = Chem.MolFromSmarts('[#6]1:[#7]:[#6]:[#6]:[#7]2:[#6]:1:[#7]:[#7]:[#6]:2')
AllChem.Compute2DCoords(s4_template)

RDLogger.DisableLog('rdApp.info')

from rdkit.Chem import PandasTools

%matplotlib inline

# Load Data

In [ ]:
dataset_morgan2 = \
    QSARDatasetIO.load(dataset_name='OSM4',
                   activity_sheetname='activity',
                   smiles_column='Canonical_Smiles',
                   id_column='OSM_ID',
                   filepath='data/osm_qsar_dataset_morgan2.xlsx',
                   calculate_similarity=False)
    

    
dataset_morgan2

In [ ]:
dataset_morgan4 = \
    QSARDatasetIO.load(dataset_name='OSM4',
                   activity_sheetname='activity',
                   smiles_column='Canonical_Smiles',
                   id_column='OSM_ID',
                   filepath='../data/osm_qsar_dataset_morgan4.xlsx',
                   calculate_similarity=False)
    
dataset_morgan4

# Load model

In [ ]:
import joblib
model_morgan2 = joblib.load("model_modsar_morgan2.joblib")

In [ ]:
results_df = pd.DataFrame(model_morgan2.cv_results_)
results_df["param_lam"] = results_df["param_lam"].astype(float)

selected_cols = [col for col in results_df.columns if 'split' not in col and 'std' not in col]

# Performance

In [ ]:
from plotnine import *

g = (ggplot(results_df, aes(x='param_lam', y="mean_fit_time")) + 
     geom_col(color="blue", fill="blue", alpha=0.6) +
     scale_x_continuous(name="Paramater $\lambda$",
                        breaks=[x/100 for x in range(21)]) +
     scale_y_continuous(name="Mean CV fit time (seconds)") +
     theme_bw() + 
     theme(figure_size = (7, 3), axis_title=element_text(size=10)) + 
     ggtitle("The higher the $\lambda$ the faster the fit time"))

g

In [ ]:
g = (ggplot(results_df, aes(x='param_lam')) + 
     geom_rect(xmin=0.025, xmax=0.065, ymin=0.5, ymax=0.75, fill='#212121', alpha=0.015) +
     geom_point(aes(y="-mean_train_neg_mean_absolute_error"), 
                size=4, color="blue", fill="blue", alpha=1) +
     geom_line(mapping=aes(y="-mean_train_neg_mean_absolute_error"), 
               size=1, color="blue", alpha=0.5) +
     geom_point(aes(y="-mean_test_neg_mean_absolute_error"), 
                size=4, color="green", fill="green", alpha=1) +
     geom_line(mapping=aes(y="-mean_test_neg_mean_absolute_error"), 
               size=1, color="green", alpha=0.5) +
     scale_x_continuous(name="Paramater $\lambda$", breaks=[x/100 for x in range(21)]) +
     scale_y_continuous(name="Mean absolute error") +
     annotate("text", label="Testing", color="green", 
              alpha=0.75, x=0.019, y=0.88, size=10, angle=1, fontweight="normal") +
     annotate("text", label="Training", color="blue", 
              alpha=0.75, x=0.012, y=0.75, size=10, angle=0, fontweight="normal") +
      annotate("text", label="Zone of best results", color="#212121", 
          alpha=0.75, x=0.045, y=0.771, size=10, fontweight="normal") +
     theme_bw() + 
     theme(figure_size = (8, 3), axis_title=element_text(size=10)))

g

In [ ]:
g.save("/mnt/data/results/mse.png")

In [ ]:
g = (ggplot(results_df, aes(x='param_lam')) + 
     geom_rect(xmin=0.025, xmax=0.065, ymin=0.8, ymax=0.92, fill='#212121', alpha=0.015) +
     geom_point(aes(y="-mean_train_neg_root_mean_squared_error"), 
                size=4, color="blue", fill="blue", alpha=1) +
     geom_line(mapping=aes(y="-mean_train_neg_root_mean_squared_error"), 
               size=1, color="blue", alpha=0.5) +
     geom_point(aes(y="-mean_test_neg_root_mean_squared_error"), 
                size=4, color="green", fill="green", alpha=1) +
     geom_line(mapping=aes(y="-mean_test_neg_root_mean_squared_error"), 
               size=1, color="green", alpha=0.5) +
     scale_x_continuous(name="Paramater $\lambda$", breaks=[x/100 for x in range(21)]) +
     scale_y_continuous(name="RMSE") +
     annotate("text", label="Testing", color="green", 
              alpha=0.75, x=0.019, y=0.88, size=10, angle=1, fontweight="normal") +
     annotate("text", label="Training", color="blue", 
              alpha=0.75, x=0.012, y=0.75, size=10, angle=0, fontweight="normal") +
      annotate("text", label="Zone of best results", color="#212121", 
          alpha=0.75, x=0.045, y=0.771, size=10, fontweight="normal") +
     theme_bw() + 
     theme(figure_size = (8, 3), axis_title=element_text(size=10)))

g

In [ ]:
g.save("/mnt/data/results/rmse.png")

In [ ]:
g = (ggplot(results_df, aes(x='param_lam')) + 
     geom_rect(xmin=0.025, xmax=0.065, ymin=-0.45, ymax=0.1, fill='#212121', alpha=0.015) +
     geom_point(aes(y="mean_train_r2"), 
                size=4, color="blue", fill="blue", alpha=1) +
     geom_line(mapping=aes(y="mean_train_r2"), 
               size=1, color="blue", alpha=0.5) +
     geom_point(aes(y="mean_test_r2"), 
                size=4, color="green", fill="green", alpha=1) +
     geom_line(mapping=aes(y="mean_test_r2"), 
               size=1, color="green", alpha=0.5) +
     scale_x_continuous(name="Paramater $\lambda$", breaks=[x/100 for x in range(21)]) +
     scale_y_continuous(name="R Square") +
     annotate("text", label="Testing", color="green", 
              alpha=0.75, x=0.019, y=0.88, size=10, angle=1, fontweight="normal") +
     annotate("text", label="Training", color="blue", 
              alpha=0.75, x=0.012, y=0.75, size=10, angle=0, fontweight="normal") +
      annotate("text", label="Zone of best results", color="#212121", 
          alpha=0.75, x=0.045, y=0.4, size=10, fontweight="normal") +
     theme_bw() + 
     theme(figure_size = (8, 3), axis_title=element_text(size=10)))

g

In [ ]:
g.save("/mnt/data/results/rmse.png")

# Graph

In [ ]:
modsar_alg = model_morgan2.best_estimator_

graph = modsar_alg.instance_graph

graph.vs['Series'] = dataset_morgan2.metadata['Series'].values

In [ ]:
module_sizes = pd.Series(graph.vs['community']).value_counts()
module_sizes

In [ ]:
import igraph 

# layout = graph.layout("kamada_kawai", maxiter=800, kkconst=5000)
#layout = graph.layout("graphopt")
layout = graph.layout('fruchterman_reingold')

In [ ]:
largest_modules = pd.Series(graph.vs["community"]).value_counts()
largest_modules

In [ ]:
import igraph
igraph.save(graph, "../data/results/model_modsar_morgan2_graph.gml")

In [ ]:
from IPython.display import display, HTML

color_dict = {"m01": "#1B9E77",
              "m02": "#D95F02",
              "m03": "#7570B3",
              "m04": "#E7298A",
              "m05": "#66A61E",
              "m06": "#A6761D",
              "m07": "#E6AB02",
              "m08": "#FFFF00"}


html_base = ("<span style=\"display: inline; background-color: %s;" 
            "padding: 10px; border: 1px solid gray;\">%s</span>")
final_html = "</p>Modules:</p>"
for module, color in color_dict.items():
    final_html += html_base % (color, module)
    
display(HTML(final_html, metadata=dict(isolated=True)))

In [ ]:
igraph.plot(graph, "model_modsar_morgan2_graph.png", 
            layout=layout, vertex_size=6, vertex_label=None, 
            vertex_color=[color_dict[comm] for comm in graph.vs["community"]],
            edge_color="#9c9a9a", edge_curved=True, edge_width=0.2,
            background=None,
            vertex_frame_width=1.3,
            bbox=[500, 300], margin=20)

# Network

In [ ]:
import networkx as nx

from nx_altair.core import to_pandas_nodes

G = nx.read_gml("model_modsar_morgan2_graph.gml")
pos = nx.kamada_kawai_layout(G, scale=0.6)

In [ ]:
molecules_df = to_pandas_nodes(G, pos)

molecules_df = pd.merge(molecules_df, 
                        dataset_morgan2.metadata[["Canonical_Smiles"]], 
                        left_index=True, 
                        right_index=True)

molecules_df.head()

In [ ]:
osm_series = molecules_df.Series
osm_series = osm_series.apply(lambda x: '4' if x == 'not4' else ('1' if x == 'TB' else x))
osm_series.name = "OSM Data"
osm_series = osm_series.apply(lambda x: 'Series '+x)

modules = molecules_df.community
modules.name = "Modules"

pd.crosstab(modules, osm_series)

# Clusters

## t-SNE

In [ ]:
from sklearn.manifold import TSNE
x_tsne = TSNE().fit_transform(dataset_morgan2.X)
x_tsne_df = pd.DataFrame(x_tsne, columns=['x1','x2'], index = dataset_morgan2.X.index)
x_tsne_df['community'] = molecules_df['community']
x_tsne_df['Series'] = molecules_df['Series']

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
tsne_communnity = sns.scatterplot(data =  x_tsne_df, x = 'x1', y = 'x2', hue = 'community')
plt.legend(bbox_to_anchor=(1.05,1),loc=2,borderaxespad=0.)
# tsne_communnity.figure.savefig("/mnt/data/results/tsne_community.png")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
tsne_series = sns.scatterplot(data =  x_tsne_df, x = 'x1', y = 'x2', hue = 'Series')
plt.legend(bbox_to_anchor=(1.05,1),loc=2,borderaxespad=0.)
# tsne_series.figure.savefig("/mnt/data/results/tsne_series.png")

## UMAP

In [ ]:
import umap
reducer = umap.UMAP()
x_umap = reducer.fit_transform(dataset_morgan2.X)

In [ ]:
x_umap_df = pd.DataFrame(x_umap, columns=['x1', 'x2'], index = dataset_morgan2.X.index)
x_umap_df['community'] = molecules_df['community']
x_umap_df['Series'] = molecules_df['Series']

In [ ]:
umap_community = sns.scatterplot(data =  x_umap_df, x = 'x1', y = 'x2', hue = 'community')
plt.legend(bbox_to_anchor=(1.05,1),loc=2,borderaxespad=0.)
umap_community.figure.savefig("/mnt/data/results/umap_community.png")

In [ ]:
umap_series = sns.scatterplot(data =  x_umap_df, x = 'x1', y = 'x2', hue = 'Series')
plt.legend(bbox_to_anchor=(1.05,1),loc=2,borderaxespad=0.)
umap_series.figure.savefig("/mnt/data/results/umap_series.png")

## network clustering

In [ ]:
modsar_community = sns.scatterplot(data =  molecules_df, x = 'x', y = 'y', hue = 'community')
plt.legend(bbox_to_anchor=(1.05,1),loc=2,borderaxespad=0.)
# modsar_community.figure.savefig("/mnt/data/results/modsar_community.png")

In [ ]:
modsar_series = sns.scatterplot(data =  molecules_df, x = 'x', y = 'y', hue = 'Series')
plt.legend(bbox_to_anchor=(1.05,1),loc=2,borderaxespad=0.)
# modsar_series.figure.savefig("/mnt/data/results/modsar_series.png")

# Analysis of Modules

In [ ]:
from rdkit.Chem.Scaffolds import rdScaffoldNetwork
from rdkit.Chem.Scaffolds import MurckoScaffold
from rdkit.Chem.Draw.MolDrawing import DrawingOptions

def get_scaffold_pattern(df, smiles_col='Canonical_Smiles'):
    params = rdScaffoldNetwork.ScaffoldNetworkParams() 
    params.includeGenericScaffolds = False
    params.includeGenericBondScaffolds = False
    params.includeScaffoldsWithoutAttachments=False
    
    smiles_mols = [smiles_code for smiles_code in df[smiles_col]]
    mols = [Chem.MolFromSmiles(smiles_code) for smiles_code in smiles_mols]
    
    net = rdScaffoldNetwork.CreateScaffoldNetwork(mols, params)
    return net

def get_mcs_pattern(df, smiles_col='Canonical_Smiles'):
    mols = [Chem.MolFromSmiles(smiles_code) for smiles_code in df[smiles_col]]
    res = rdFMCS.FindMCS(mols)
    pattern = Chem.MolFromSmarts(res.smartsString)
    return pattern

def get_representative_scaffold(module_df, smiles_col='Canonical_Smiles'):
    net = get_scaffold_pattern(module_df, smiles_col)

    df = pd.DataFrame({'pattern': net.nodes, 'count': net.counts})
    df['pctg'] = df["count"].apply(lambda x: x/module_df.shape[0])
    df = df[(df["pctg"] < 1) & (df["pctg"] > 0.5)].copy()
    df['pctg'] = df['pctg'].apply(lambda x: "%.2f %%" % (100*x))

    def generate_depiction(x):
        m = Chem.MolFromSmiles(x)
        if m.HasSubstructMatch(s4_template):
            AllChem.GenerateDepictionMatching2DStructure(m, s4_template)
        return m

    df["scaffold"] = df["pattern"].apply(generate_depiction)
    
#     PandasTools.AddMoleculeColumnToFrame(df,'pattern','scaffold',includeFingerprints=True)
    return df.sort_values('count', ascending=False)

def plot_representative_of_module(df, selected_module, 
                                  smiles_col='Canonical_Smiles', align_pattern=None, 
                                  folder="/mnt/data/results/OSM_model_round2/"):
    
    module_df = df[df['community'] == selected_module]
    
    module_subgraph = G.subgraph([node[0] for node in G.nodes(data=True) 
                                  if node[1]['community'] == selected_module])
    degrees = pd.DataFrame(sorted(module_subgraph.degree(), key=lambda x: x[1], reverse=True), 
                           columns=["index","node_degree"])
    degrees.set_index("index", inplace=True)
    
    idx = 0
    while idx < len(degrees):
        try:
            highest_degree_id = degrees.index[idx]
            #highest_degree_id = module_df['degree'].astype(int).idxmax()
            print("Representative molecule of module {} selected: {}".format(selected_module, highest_degree_id))
            highest_degree_mol = Chem.MolFromSmiles(module_df.loc[highest_degree_id]['Canonical_Smiles'])
            print("Canonical Smiles: ", module_df.loc[highest_degree_id]['Canonical_Smiles'])

            module_pattern = get_representative_scaffold(module_df, smiles_col='Canonical_Smiles')
            module_pattern = Chem.MolFromSmarts(module_pattern["pattern"].iloc[0])

            if align_pattern is not None:
                AllChem.Compute2DCoords(align_pattern)
                AllChem.GenerateDepictionMatching2DStructure(highest_degree_mol, align_pattern)

            matching = highest_degree_mol.GetSubstructMatch(module_pattern)
            hit_bonds = []
            for bond in module_pattern.GetBonds():
                aid1 = matching[bond.GetBeginAtomIdx()]
                aid2 = matching[bond.GetEndAtomIdx()]
                hit_bonds.append(highest_degree_mol.GetBondBetweenAtoms(aid1,aid2).GetIdx())
            break
        except Exception as e:
            print(e)
            idx = idx + 1
        
        
    drawing_opts = DrawingOptions()
    drawing_opts.bgColor=None
    drawing_opts.colorBonds = False
    drawing_opts.includeAtomNumbers = True
    
    filename = '%s/%s_%s_pattern.png' % (folder, selected_module, highest_degree_id)
    print("Saving %s" % filename)
    Chem.Draw.MolToFile(highest_degree_mol, filename, size=(400, 300),
                        highlightAtoms=matching, highlightBonds=hit_bonds, 
                        kekulize=True,
                        options=drawing_opts)
    
    return  Chem.Draw.MolToImage(highest_degree_mol,
                     size=(400, 300), kekulize=True, 
                     highlightAtoms=matching, highlightBonds=hit_bonds,
                     options=drawing_opts,)

## Module 1

In [ ]:
module_pattern1 = get_representative_scaffold(molecules_df[molecules_df["community"] == "m01"], 
                                             smiles_col='Canonical_Smiles')

module_pattern1

In [ ]:
module_pattern1['scaffold'].iloc[0]

In [ ]:
from rdkit.Chem.Draw import MolToImageFile
MolToImageFile(module_pattern1['scaffold'].iloc[0], "/mnt/data/results/m01_pattern.png")

In [ ]:
plot_representative_of_module(molecules_df, selected_module='m01')

In [ ]:
m1_pattern = get_representative_scaffold(molecules_df[molecules_df['community'] == "m01"])
Chem.MolFromSmarts(m1_pattern["pattern"].values[0])

In [ ]:
molecules_df[molecules_df["community"] == "m01"].groupby('Series').count()

## Module 2

In [ ]:
module_pattern2 = get_representative_scaffold(molecules_df[molecules_df["community"] == "m02"], 
                                             smiles_col='Canonical_Smiles')

module_pattern2

In [ ]:
module_pattern2['scaffold'].iloc[0]

In [ ]:
MolToImageFile(module_pattern2['scaffold'].iloc[0], "/mnt/data/results/m02_pattern.png")

In [ ]:
plot_representative_of_module(molecules_df, selected_module='m02')

In [ ]:
m2_pattern = get_representative_scaffold(molecules_df[molecules_df['community'] == "m02"])
Chem.MolFromSmarts(m2_pattern["pattern"].values[0])

In [ ]:
molecules_df[molecules_df["community"] == "m02"].groupby('Series').count()

## Module 3

In [ ]:
module_pattern3 = get_representative_scaffold(molecules_df[molecules_df["community"] == "m03"], 
                                             smiles_col='Canonical_Smiles')

module_pattern3

In [ ]:
module_pattern3['scaffold'].iloc[0]

In [ ]:
MolToImageFile(module_pattern3['scaffold'].iloc[0], "/mnt/data/results/m03_pattern0.png")

In [ ]:
plot_representative_of_module(molecules_df, selected_module='m03')

In [ ]:
m3_pattern = get_representative_scaffold(molecules_df[molecules_df['community'] == "m03"])
Chem.MolFromSmarts(m3_pattern["pattern"].values[0])

In [ ]:
molecules_df[molecules_df["community"] == "m03"].groupby('Series').count()

## Module 4

In [ ]:
module_pattern4 = get_representative_scaffold(molecules_df[molecules_df["community"] == "m04"], 
                                             smiles_col='Canonical_Smiles')

module_pattern4

In [ ]:
module_pattern4['scaffold'].iloc[0]

In [ ]:
MolToImageFile(module_pattern4['scaffold'].iloc[0], "/mnt/data/results/m04_pattern0.png")

In [ ]:
plot_representative_of_module(molecules_df, selected_module='m04')

In [ ]:
m4_pattern = get_representative_scaffold(molecules_df[molecules_df['community'] == "m04"])
Chem.MolFromSmarts(m4_pattern["pattern"].values[0])

In [ ]:
molecules_df[molecules_df["community"] == "m04"].groupby('Series').count()

## Module 5

In [ ]:
module_pattern5 = get_representative_scaffold(molecules_df[molecules_df["community"] == "m05"], 
                                             smiles_col='Canonical_Smiles')

module_pattern5

In [ ]:
module_pattern5['scaffold'].iloc[0]

In [ ]:
MolToImageFile(module_pattern5['scaffold'].iloc[0], "/mnt/data/results/m05_pattern0.png")

In [ ]:
m5_pattern = get_representative_scaffold(molecules_df[molecules_df['community'] == "m05"])
Chem.MolFromSmarts(m5_pattern["pattern"].values[0])

In [ ]:
plot_representative_of_module(molecules_df, selected_module='m05')

In [ ]:
molecules_df[molecules_df["community"] == "m05"].groupby('Series').count()

# Piecewise Model

In [ ]:
def get_linear_equation(row):
    coeffs = ['%+.2f %s' % (val, coeff) for coeff, val in row.iteritems() 
              if not (math.isnan(val) or float(val) == 0) and coeff != 'B']
    equation_str = ' '.join(coeffs)
    equation_str += " %+.2f" % row['B']    
    return equation_str  

def print_module_equations(coeffs, module):
    """Helper function to print a clean representation of piecewise equations for informed module"""
    
    coeffs = coeffs.query('module == "%s"' % module)

    for idx, row in coeffs.groupby(['module', 'region']):
        equation = get_linear_equation(row.drop(columns=['module', 'region']))
        print('Equations for Module %s | Region %d\n' % (idx[0], idx[1] + 1))
        print("pIC50 = ", equation)
        print()

In [ ]:
coeff, breakpoints = modsar_alg.get_model_info()

In [ ]:
coeff.sort_values(by="module")

## Linear Equations

In [ ]:
print_module_equations(coeff, 'm01')

In [ ]:
print_module_equations(coeff, 'm02')

In [ ]:
print_module_equations(coeff, 'm03')

In [ ]:
print_module_equations(coeff, 'm04')

In [ ]:
print_module_equations(coeff, 'm05')

## Breakpoints

In [ ]:
def draw_ecfp_bit(bit_name, df, fps):
    ones_df = df.loc[fps[fps[bit_name] == 1].index]
    mol = Chem.MolFromSmiles(ones_df['Canonical_Smiles'].iloc[1])
    bi = {}
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=1024, bitInfo=bi)
    bit = int(bit_name.replace('Bit_',''))
    return mol,bit,bi

#
# Functions for providing detailed descriptions of MFP bits from Nadine Schneider 
#  It's probably better to do this using the atomSymbols argument but this does work.
#
def includeRingMembership(s, n):
    r=';R]'
    d="]"
    return r.join([d.join(s.split(d)[:n]),d.join(s.split(d)[n:])])
 
def includeDegree(s, n, d):
    r=';D'+str(d)+']'
    d="]"
    return r.join([d.join(s.split(d)[:n]),d.join(s.split(d)[n:])])
 
def writePropsToSmiles(mol,smi,order):
    #finalsmi = copy.deepcopy(smi)
    finalsmi = smi
    for i,a in enumerate(order):
        atom = mol.GetAtomWithIdx(a)
        if atom.IsInRing():
            finalsmi = includeRingMembership(finalsmi, i+1)
        finalsmi = includeDegree(finalsmi, i+1, atom.GetDegree())
    return finalsmi
 
def getSubstructSmi(mol,atomID,radius):
    if radius>0:
        env = Chem.FindAtomEnvironmentOfRadiusN(mol,radius,atomID)
        atomsToUse=[]
        for b in env:
            atomsToUse.append(mol.GetBondWithIdx(b).GetBeginAtomIdx())
            atomsToUse.append(mol.GetBondWithIdx(b).GetEndAtomIdx())
        atomsToUse = list(set(atomsToUse))
    else:
        atomsToUse = [atomID]
        env=None
    smi = Chem.MolFragmentToSmiles(mol,atomsToUse,bondsToUse=env,allHsExplicit=True, allBondsExplicit=True, rootedAtAtom=atomID)
    order = eval(mol.GetProp("_smilesAtomOutputOrder"))
    smi2 = writePropsToSmiles(mol,smi,order)
    return smi,smi2,atomsToUse,env

In [ ]:
breakpoints[breakpoints["module"] == "m01"]

In [ ]:
mol,bit,bi = draw_ecfp_bit('Bit_0350', molecules_df,dataset_morgan2.X)
bit350 = Draw.DrawMorganBit(mol,bit,bi)
bit350

In [ ]:
mol,bit,bi = draw_ecfp_bit('Bit_0350', molecules_df,dataset_morgan2.X)
atomid = bi[bit][0][0]
r = bi[bit][0][1]
_, sma,hit_ats,hit_bonds = getSubstructSmi(mol,atomid,r)
sma

In [ ]:
bit350.save("/mnt/data/results/bit_350.png")

In [ ]:
breakpoints[breakpoints["module"] == "m02"]

In [ ]:
mol,bit,bi = draw_ecfp_bit('Bit_0875', molecules_df,dataset_morgan2.X)
bit875 = Draw.DrawMorganBit(mol,bit,bi)
bit875

In [ ]:
mol,bit,bi = draw_ecfp_bit('Bit_0875', molecules_df,dataset_morgan2.X)
bit875 = Draw.DrawMorganBit(mol,bit,bi)
bit875.save("/mnt/data/results/bit_875.png")
atomid = bi[bit][0][0]
r = bi[bit][0][1]
_, sma,hit_ats,hit_bonds = getSubstructSmi(mol,atomid,r)
sma

In [ ]:
bit875.save("/mnt/data/results/bit_875.png")

In [ ]:
breakpoints[breakpoints["module"] == "m03"]

In [ ]:
mol,bit,bi = draw_ecfp_bit('Bit_0896', molecules_df,dataset_morgan2.X)
bit896 = Draw.DrawMorganBit(mol,bit,bi)
# bit896.save("/mnt/data/results/bit_896.png")
bit896

In [ ]:
atomid = bi[bit][0][0]
r = bi[bit][0][1]
_, sma,hit_ats,hit_bonds = getSubstructSmi(mol,atomid,r)
sma

In [ ]:
breakpoints[breakpoints["module"] == "m04"]

In [ ]:
mol,bit,bi = draw_ecfp_bit('Bit_0484', molecules_df,dataset_morgan2.X)
bit484 = Draw.DrawMorganBit(mol,bit,bi)
# bit484.save("/mnt/data/results/bit_484.png")
bit484

In [ ]:
atomid = bi[bit][0][0]
r = bi[bit][0][1]
_, sma,hit_ats,hit_bonds = getSubstructSmi(mol,atomid,r)
sma

In [ ]:
breakpoints[breakpoints["module"] == "m05"]

In [ ]:
mol,bit,bi = draw_ecfp_bit('Bit_0248', molecules_df,dataset_morgan2.X)
bit248 = Draw.DrawMorganBit(mol,bit,bi)
# bit248.save("/mnt/data/results/bit_248.png")
bit248

In [ ]:
atomid = bi[bit][0][0]
r = bi[bit][0][1]
_, sma,hit_ats,hit_bonds = getSubstructSmi(mol,atomid,r)
sma

# Applicability Domain

## Train/test set split
Any method and any data split could be used here, we selected the seventh data split at random state = 0.

In [ ]:
import joblib
import matplotlib.pyplot as plt

In [ ]:
cv_results = joblib.load("data/results/modsar_alg_cv_result5_0.joblib")
cv_results_df = pd.DataFrame(cv_results)
cv_results_df

In [ ]:
modsar_alg_cv = cv_results_df['estimator'].iloc[1]

In [ ]:
from sklearn.model_selection import ShuffleSplit
cv = ShuffleSplit(n_splits=5, test_size=0.1, random_state=0)

In [ ]:
train_meta = dataset_morgan2.metadata.iloc[list(cv.split(dataset_morgan2.X))[1][0]]
test_meta = dataset_morgan2.metadata.iloc[list(cv.split(dataset_morgan2.X))[1][1]]

In [ ]:
X_train = dataset_morgan2.X.iloc[list(cv.split(dataset_morgan2.X))[1][0]]
X_test = dataset_morgan2.X.iloc[list(cv.split(dataset_morgan2.X))[1][1]]
y_train = dataset_morgan2.y.iloc[list(cv.split(dataset_morgan2.X))[1][0]]
y_test = dataset_morgan2.y.iloc[list(cv.split(dataset_morgan2.X))[1][1]]

## Leverage Approach and Williams Plot

In [ ]:
import seaborn as sns

In [ ]:
def hat_matrix(X1):#, X2): #Hat Matrix
    hat_mat =  np.dot(np.dot(X1, np.linalg.inv(np.dot(X1.T, X1))), X1.T)
    return hat_mat
    
def williams_plot(X_train, X_test, Y_true_train, Y_true_test, y_pred_train, y_pred_test, toPrint = True,toPlot=True,path = './',filename = ''):
    H_train= hat_matrix(np.concatenate([X_train, X_test], axis=0))#, numpy.concatenate([X_train, X_test], axis=0))
#     y_pred_train= model.predict(X_train)
#     y_pred_test= model.predict(X_test)
    
    y_pred_test = y_pred_test.reshape(y_pred_test.shape[0],)
    y_pred_train = y_pred_train.reshape(y_pred_train.shape[0],)
    Y_true_train = Y_true_train.reshape(Y_true_train.shape[0],)
    Y_true_test = Y_true_test.reshape(Y_true_test.shape[0],)
    
    residual_train= np.abs(Y_true_train - y_pred_train)
    residual_test= np.abs(Y_true_test - y_pred_test)
    s_residual_train = ((residual_train) - np.mean(residual_train)) / np.std(residual_train)
    s_residual_test = (residual_test - np.mean(residual_test))/ np.std(residual_test)

    leverage= np.diag(H_train)
    leverage_train = leverage[0:X_train.shape[0]]
    leverage_test = leverage[X_train.shape[0]:]
    p = X_train.shape[1] #features
    n = X_train.shape[0] #+ X_test.shape[0] #training compounds
    h_star = (3 * (p+1))/float(n)
    
    train_points_in_ad = float(100 * np.sum(np.asarray(leverage_train < h_star) & np.asarray(s_residual_train<3))) / len(leverage_train)
    test_points_in_ad = float(100 * np.sum(np.asarray(leverage_test < h_star) & np.asarray(s_residual_test<3))) / len(leverage_test)

    test_lev_out = np.sum(np.asarray(leverage_test > h_star))
    
    if toPrint:
        print("Percetege of train points inside AD: {}%".format(train_points_in_ad))
        print("Percetege of test points inside AD: {}%".format(test_points_in_ad))
        print("h*: {}".format(h_star))
        
    train_df = pd.DataFrame({"x":leverage_train.tolist(), "y": s_residual_train.tolist()})
    train_df['label'] = 'train'
    test_df = pd.DataFrame({"x":leverage_test.tolist(), "y": s_residual_test.tolist()})
    test_df['label'] = 'test'
    df = pd.concat([train_df, test_df], axis = 0)

    if toPlot:
    
#       plt.plot(leverage_train.tolist(),s_residual_train.tolist(),'o', label='train')
#         sns.scatterplot(leverage_train.tolist(),s_residual_train.tolist(), label='train')
#       plt.plot(leverage_test.tolist(),s_residual_test.tolist(),'^', label = 'test')
#         sns.scatterplot(leverage_test.tolist(),s_residual_test.tolist(), label = 'test')
        sns.scatterplot(df, x = "x", y = "y", hue = 'label')
        plt.axhline(y=3, linestyle='--')
        plt.axhline(y=-3, linestyle='--')
        plt.axvline(x=h_star, linestyle='-')
        plt.text(x = 0.0263, y = 3.3, s = 'h*', color = '#1f77b4')
        plt.ylim(bottom=-3.2)
        plt.xlabel('Leverage')
        plt.ylabel('Standardized Residuals')
        plt.legend(loc='lower right', shadow=True)
        plt.savefig("data/williams_plot2.png")
#       plt.close()
        plt.show()

    return test_points_in_ad,train_points_in_ad,test_lev_out,h_star,leverage_train,leverage_test,s_residual_train,s_residual_test

In [ ]:
from sklearn.decomposition import PCA
y_train_pred = modsar_alg_cv.predict(X_train)
y_test_pred = modsar_alg_cv.predict(X_test)

In [ ]:
pca_model1 = PCA(n_components = 2)
all_data = pca_model1.fit_transform(dataset_morgan2.X)
x_train_pca = all_data[list(cv.split(dataset_morgan2.X))[1][0]]
x_test_pca = all_data[list(cv.split(dataset_morgan2.X))[1][1]]

In [ ]:
wp = williams_plot(x_train_pca, x_test_pca, y_train.values, y_test.values, y_train_pred, y_test_pred)

# SHAP value

In [ ]:
from sklearn.linear_model import LinearRegression
import shap
import shap_barplot
from shap_barplot import barplot

In [ ]:
explainer = shap.Explainer(modsar_alg.models['m01'].predict, dataset_morgan2.X.loc[molecules_df[molecules_df['community'] == 'm01'].index])
shap_values1 = explainer(dataset_morgan2.X.loc[molecules_df[molecules_df['community'] == 'm01'].index])
shap.plots.waterfall(shap_values1[0], max_display=12)

In [ ]:
shap.plots.bar(shap_values1, max_display=13)

In [ ]:
column_names = dataset_morgan2.X.columns
img,_ = barplot.shap_barplot(dataset_morgan2.X.loc[molecules_df[molecules_df['community'] == 'm01'].index], shap_values1.values, column_names,num_features=12)

In [ ]:
shap.plots.heatmap(shap_values1, max_display=13)

In [ ]:
explainer2 = shap.Explainer(modsar_alg.models['m02'].predict, dataset_morgan2.X.loc[molecules_df[molecules_df['community'] == 'm02'].index])
shap_values2 = explainer2(dataset_morgan2.X.loc[molecules_df[molecules_df['community'] == 'm02'].index])
shap.plots.waterfall(shap_values2[0], max_display=3)

In [ ]:
shap.plots.bar(shap_values2, max_display=4)

In [ ]:
column_names = dataset_morgan2.X.columns
img,_ = barplot.shap_barplot(dataset_morgan2.X.loc[molecules_df[molecules_df['community'] == 'm02'].index], shap_values2.values, column_names,num_features=3)

In [ ]:
shap.plots.heatmap(shap_values2, max_display=4)

In [ ]:
explainer3 = shap.Explainer(modsar_alg.models['m03'].predict, dataset_morgan2.X.loc[molecules_df[molecules_df['community'] == 'm03'].index])
shap_values3 = explainer3(dataset_morgan2.X.loc[molecules_df[molecules_df['community'] == 'm03'].index])
shap.plots.waterfall(shap_values3[0], max_display= 4)

In [ ]:
shap.plots.bar(shap_values3,max_display= 4)

In [ ]:
column_names = dataset_morgan2.X.columns
img,_ = barplot.shap_barplot(dataset_morgan2.X.loc[molecules_df[molecules_df['community'] == 'm03'].index], shap_values3.values, column_names,num_features=3)

In [ ]:
shap.plots.heatmap(shap_values3, max_display=4)

In [ ]:
explainer4 = shap.Explainer(modsar_alg.models['m04'].predict, dataset_morgan2.X.loc[molecules_df[molecules_df['community'] == 'm04'].index])
shap_values4 = explainer4(dataset_morgan2.X.loc[molecules_df[molecules_df['community'] == 'm04'].index])
shap.plots.waterfall(shap_values4[0], max_display=3)

In [ ]:
shap.plots.bar(shap_values4, max_display=3)

In [ ]:
column_names = dataset_morgan2.X.columns
img,_ = barplot.shap_barplot(dataset_morgan2.X.loc[molecules_df[molecules_df['community'] == 'm04'].index], shap_values4.values, column_names,num_features=2)

In [ ]:
shap.plots.heatmap(shap_values4, max_display=3)

In [ ]:
explainer5 = shap.Explainer(modsar_alg.models['m05'].predict, dataset_morgan2.X.loc[molecules_df[molecules_df['community'] == 'm05'].index])
shap_values5 = explainer5(dataset_morgan2.X.loc[molecules_df[molecules_df['community'] == 'm05'].index])
shap.plots.waterfall(shap_values5[0], max_display=12)

In [ ]:
shap.plots.bar(shap_values5, max_display=13)

In [ ]:
column_names = dataset_morgan2.X.columns
img,_ = barplot.shap_barplot(dataset_morgan2.X.loc[molecules_df[molecules_df['community'] == 'm05'].index], shap_values5.values, column_names,num_features=15)

In [ ]:
shap.plots.heatmap(shap_values5, max_display=13)

# Visualize Important Bits

In [ ]:
from rdkit.Chem.Draw import rdMolDraw2D
from IPython.display import SVG
def show_compound_example(bits_names, compound_name,positive = True):
    mol = Chem.MolFromSmiles(molecules_df.loc[compound_name]['Canonical_Smiles'])
    bi = {}
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=1024, bitInfo=bi)
    bits = [int(b.replace('Bit_','')) for b in bits_names]
    atomids = [bi[b][0][0] for b in bits]
    rs = [bi[b][0][1] for b in bits]
    hit_ats = []
    hit_bonds = []
    for i in range(len(bits)):
        _, _, tmp_ats, tmp_bonds = getSubstructSmi(mol,atomids[i],rs[i])
        for j in tmp_ats:
            if j not in hit_ats:
                hit_ats.append(j)
        if tmp_bonds is not None:
            for s in tmp_bonds:
                if s not in hit_bonds:
                    hit_bonds.append(s)

    if positive == True:
        colors = (0.8,0.0,0.8)
    else:
        colors = (0,0,0.8)
    atom_cols = {}
    for at in hit_ats:
        atom_cols[at] = colors

    bond_cols = {}
    for bd in hit_bonds:
        bond_cols[bd] = colors

    d = rdMolDraw2D.MolDraw2DSVG(200, 200)
    rdMolDraw2D.PrepareAndDrawMolecule(d, mol, highlightAtoms=hit_ats,
                                       highlightAtomColors=atom_cols,
                                       highlightBonds=hit_bonds,
                                       highlightBondColors=bond_cols)
    d.FinishDrawing()
    svg = d.GetDrawingText().replace('svg:','')
    return svg

## Module 1

In [ ]:
print('Bit_0290 in Compound OSM-S-109')
SVG(show_compound_example(['Bit_0290'],'OSM-S-109'))

In [ ]:
import rdkit
from rdkit import Chem
Chem.MolFromSmiles('Cc1cc(/C=C2\S/C(=N\c3ccccc3)NC2=O)c(C)n1-c1ccc([N+](=O)[O-])cc1')

In [ ]:
print('Bit_0745 in Compound OSM-S-110')
SVG(show_compound_example(['Bit_0745'],'OSM-S-110', False))

In [ ]:
print('Bit_0350 in Compound OSM-S-106')
SVG(show_compound_example(['Bit_0350'],'OSM-S-106', False))

## Module 2, 3, 5

In [ ]:
print('Bit_0890 in Compound OSM-S-412')
SVG(show_compound_example(['Bit_0890'],'OSM-S-412'))

In [ ]:
print('Series 4 core in Compound OSM-S-189')
SVG(show_compound_example(['Bit_0890','Bit_0711','Bit_0248','Bit_0819','Bit_0399'],'OSM-S-189'))

In [ ]:
print('Bit_0896 in Compound OSM-S-189')
SVG(show_compound_example(['Bit_0896'],'OSM-S-189'))

# Important Bits for Series 4

## Get SMARTS from Bit

In [ ]:
mol,bit,bi = draw_ecfp_bit('Bit_0890', molecules_df,dataset_morgan2.X)
atomid = bi[bit][0][0]
r = bi[bit][0][1]
_, sma,hit_ats,hit_bonds = getSubstructSmi(mol,atomid,r)
sma

In [ ]:
from rdkit.Chem import MolFromSmarts
from rdkit.Chem import MolToSmiles

In [ ]:
MolToSmiles(MolFromSmarts('[n;R;D3](:[c;R;D3](:[cH;R;D2])-[O;D2])(:[c;R;D3](-[c;R;D3]):[n;R;D2]):[c;R;D3](:[cH;R;D2]):[n;R;D2]'))

In [ ]:
mol,bit,bi = draw_ecfp_bit('Bit_0896', molecules_df,dataset_morgan2.X)
atomid = bi[bit][0][0]
r = bi[bit][0][1]
_, sma,hit_ats,hit_bonds = getSubstructSmi(mol,atomid,r)
sma

In [ ]:
MolToSmiles(MolFromSmarts('[cH;R;D2](:[cH;R;D2]:[c;R;D3]):[c;R;D3](-[c;R;D3]):[cH;R;D2]'))